In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from random import randint
from sklearn.preprocessing import MinMaxScaler

import keras
from keras.models import Sequential
from keras.layers import Activation, Dropout
from keras.layers.core import Dense, Flatten
#from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
#from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPool2D
from keras.utils.layer_utils import count_params
from keras.utils import np_utils


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

import tensorflow as tf

In [ ]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
y_train = np.array(train.label)
X_train = np.array(train.drop(labels = ["label"],axis = 1)).reshape(-1,28,28,1)
X_test = np.array(test).reshape(-1,28,28,1)
print(X_train.shape,X_test.shape)

In [ ]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [ ]:
plt.imshow(X_train[5].reshape(28,28), cmap=plt.cm.binary)
plt.show()
print('Label: {}'.format(y_train[5]))

In [ ]:
model = Sequential()

# first conv layer
model.add(Conv2D(128,(3,3),activation='relu',input_shape = X_train.shape[1:]))
model.add(MaxPool2D(pool_size=(2,2)))


# 2nd conv layer
model.add(Conv2D(64,(3,3),activation='relu')) 
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# first connected layer
model.add(Flatten())  
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.25))

# 2nd connected layer
model.add(Dense(10)) 
model.add(Activation("softmax")) # softmax ac fun for classification


In [ ]:
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
y_train = np_utils.to_categorical(y_train)

In [ ]:
model.fit(x = X_train, y = y_train, validation_split=0.25, epochs=10, shuffle=True, verbose=2)

In [ ]:
predictions = model.predict(x=X_train, verbose=0) 

In [ ]:
predictions_classes = [np.argmax(predictions[i]) for i in range(0, len(predictions))]

In [ ]:
y_train = [np.argmax(y_train[i]) for i in range(0, len(y_train))]

In [ ]:
cm = confusion_matrix(y_true=y_train, y_pred=predictions_classes)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = [i for i in range(0,10)]
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
prediction_test = model.predict(x=X_test, verbose=0)

In [ ]:
sample_submissions = [np.argmax(prediction_test[i]) for i in range(0, len(prediction_test))]


In [ ]:
submit=pd.read_csv('../input/digit-recognizer/sample_submission.csv', header='infer')
submit["Label"]=sample_submissions
submit.to_csv('submission.csv', index=False)